In [1]:
import numpy as np
import binascii

In [3]:
def decimalToBinary(num , places):
    return ('{0:0' +  str(places) + 'b}').format(num)


def binaryToDecimal(binary):
    return int(binary, 2)
        
def xorBinary(binary,key = "00000000"):
    i=0
    encBin = ""
    while(i<8):
        if(binary[i] == '0' and key[i] == '0') or (binary[i] == '1' and key[i] == '1'):
            encBin+="0"
        else:
            encBin+="1"
        i+=1
        
#     print(binary , key , encBin)    
    return encBin
    
def compressBinary(binary):
    i=0
    compressBin = ""
    while(i<8):
        if(i%2 == 0):
            if((binary[i] == '0' and binary[i+1] == '0') or (binary[i] == '1' and binary[i+1] == '1')):
                compressBin+="0"
            else:
                compressBin+="1"
        i+=1
        
    return compressBin

def to_twoscomplement(bits, value):
    if value < 0:
        value = ( 1<<bits ) + value
    formatstring = '{:0%ib}' % bits
    return formatstring.format(value)

# Code for decoding

In [4]:
import sys, traceback
import time , socket

In [7]:
def server_program():
    # get the hostname
    host = '192.168.43.136'
    port = 5000  # initiate port no above 1024
    count = 0
    current = 0
    currentByte = ''
    last = 0
    org_data = []
    start_time = None
    end_time = None

    
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # get instance
    server_socket.bind((host, port))  # bind host address and port together
    server_socket.listen(1)

    try :
        connection,client_address = server_socket.accept()
        while True:
            data = connection.recv(1)
            if not data:
                end_time = int(time.time() * 1000)
                print("Time in MS :" , end_time - start_time)
                print("Packet Transfered :" , count)
                connection.close()
                server_socket.close()
                return count , (end_time - start_time)
            
            if data[0] == '0':
                current+=1
                currentByte+=data[1:]
                if current == 3:
                    dataPoint = int(currentByte[1:],2)
                    if currentByte[0] == '1':
                        dataPoint *= -1
                    org_data.append(dataPoint)
                    last=dataPoint
                    current = 0
                    currentByte = ''
                    
            else:
                dataPoint = int(data[2:],2)
                if data[1]=='1':
                    dataPoint*=-1
                dataPoint+=last
                org_data.append(dataPoint)
                last = dataPoint
        
    except Exception:
        print("Some Error Occured")
        traceback.print_exc()
        connection.close()
        server_socket.close()

# Code for Un-compressed

In [9]:
def UC_server_program():
    # get the hostname
    host = '192.168.43.136'
    port = 5000  # initiate port no above 1024
    count = 0
    current = 0
    currentByte = ''
    org_data = []
    start_time = None
    end_time = None

    
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # get instance
    server_socket.bind((host, port))  # bind host address and port together
    server_socket.listen(1)

    try :
        connection,client_address = server_socket.accept()
        while True:
            data = connection.recv(1)
            if not data:
                end_time = int(time.time() * 1000)
                print("Time in MS :" , end_time - start_time)
                print("Packet Transfered :" , count)
                connection.close()
                server_socket.close()
                return count , (end_time - start_time)
            
            else:
                current+=1
                currentByte+=data[1:]
                if current == 3:
                    dataPoint = int(currentByte[1:],2)
                    if currentByte[0] == '1':
                        dataPoint *= -1
                    org_data.append(dataPoint)
                    current = 0
                    currentByte = ''

    except Exception:
        print("Some Error Occured")
        traceback.print_exc()
        connection.close()
        server_socket.close()